In [60]:
%pip install ultralytics
%pip install super-gradients
%pip install supervision
%pip install torch

In [61]:
from ultralytics import NAS

In [62]:
from google.colab import drive
drive.mount('/content/drive')

In [63]:
from super_gradients.training import models
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"

best_model = models.get(
    'yolo_nas_l',
    num_classes=1,
    checkpoint_path="/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/average_model.pth"
).to(DEVICE)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[2024-01-03 23:32:30] INFO - checkpoint_utils.py - Successfully loaded model weights from /content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/average_model.pth EMA checkpoint.


In [33]:
import supervision as sv
import ultralytics

def get_inference(path_project):
  ds = sv.DetectionDataset.from_yolo(
      images_directory_path=f"{path_project}/images",
      annotations_directory_path=f"{path_project}/labels",
      data_yaml_path=f"/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/data.yaml",
      force_masks=False
  )

  CONFIDENCE_TRESHOLD = 0.5
  predictions = {}
  for image_name, image in ds.images.items():
    result = list(best_model.predict(image, conf=CONFIDENCE_TRESHOLD))[0]
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int)
    )
    predictions[image_name] = detections
    nome_imagem = image_name.split('/')[7] + ".txt"
    with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/inferencias/{nome_imagem}","w") as file:
      file.write(str(detections.xyxy))

get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/test")
get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train")
get_inference("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid")

[2024-01-03 22:24:53] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 22:24:53] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 22:24:53] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 22:24:53] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 22:24:54] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 22:24:54] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 22:24:54] INFO -

In [64]:
import supervision as sv
import ultralytics
box_annotator = sv.BoxAnnotator()

images = []
titles = []

detections_list = []
prediction_list = []

def get_ioudata(path_project):
  ds = sv.DetectionDataset.from_yolo(
      images_directory_path=f"{path_project}/images",
      annotations_directory_path=f"{path_project}/labels",
      data_yaml_path=f"/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/data.yaml",
      force_masks=False
  )
  CONFIDENCE_TRESHOLD = 0.5
  predictions = {}
  for image_name, image in ds.images.items():
    result = list(best_model.predict(image, conf=CONFIDENCE_TRESHOLD))[0]
    detections = sv.Detections(
        xyxy=result.prediction.bboxes_xyxy,
        confidence=result.prediction.confidence,
        class_id=result.prediction.labels.astype(int)
    )
    predictions[image_name] = detections

  keys = list(ds.images.keys())
  for key in keys:
      # Para anotações
      annotation_detections = ds.annotations[key]
      for detection in annotation_detections:
          detections_list.append(detection[0])

      # Para previsões
      prediction_detections = predictions[key]
      prediction_list.append(prediction_detections)

get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/test")
get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train")
get_ioudata("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid")

[2024-01-03 23:32:31] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 23:32:31] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 23:32:31] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 23:32:32] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 23:32:32] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 23:32:33] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-03 23:32:33] INFO -

In [73]:
import numpy as np


def iou_xyxy(detections, predictions):

  def iou(box1,box2):
    # Check if the bounding boxes are valid.
    if box1[2] is None or box2[2] is None:
      return 0.0

    # Calculate the intersection area.
    inter_top = max(box1[1], box2[1])
    inter_bottom = min(box1[3], box2[3])
    inter_left = max(box1[0], box2[0])
    inter_right = min(box1[2], box2[2])
    inter_area = (inter_right - inter_left) * (inter_bottom - inter_top)

    # Calculate the union area.
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area

    # Calculate the IoU.
    iou = inter_area / union_area
    return iou


  for box2 in predictions:
    return iou(detections,box2[0])


for i in range(len(detections_list)):
  iou_score = iou_xyxy(detections_list[i], prediction_list[i])
  with open("/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/Train6YoloNAS/Iou.txt",'a') as file:
    file.write(str(iou_score)+ '\n')
  print(iou_score)

0.7692672
0.96136683
0.9186375
0.8434799
0.9056891
0.8446695
0.7916934
0.8845069
0.83443916
None
0.90673774
0.8727609
0.87802154
0.8654502
0.7298849
0.85014606
0.8266967
0.8986989
0.9099375
0.93238854
0.8324876
0.8888012
0.88084704
0.8464839
0.97059596
0.4547523
None
0.8763265
0.69546
0.8760197
0.9549727
0.94291633
0.9573195
0.9511287
0.9687374
0.9564663
0.9710045
0.97588325
0.9678368
0.9551399
0.95759493
0.971605
0.95439875
0.9413378
0.93179923
0.9585048
0.9812537
0.95734483
0.9634278
1.0300786
0.51886725
0.057182785
-0.10625204
0.038105153
0.22348829
-0.086570546
0.07582873
-0.01546494
-0.42905545
0.061034545
0.2228453
-0.14966732
-0.29209548
-0.07101269
-0.060751937
-0.25234953
-0.026268763
-0.63948554
-1.349359
0.5492155
-0.30988666
-0.05474178
0.06701056
0.27279913
0.36352593
-0.011361393
0.09913011
-0.24355944
-0.22915366
0.4468323
-1.2686417
0.38362375
0.041297205
-0.2364475
0.10959251
-1.6656244
0.030053005
-0.19622147
0.033232898
-0.18318455
3.779119
0.068785354
-0.38470796
0.

IndexError: ignored

In [19]:
def box_iou_batch(boxes_true: np.ndarray, boxes_detection: np.ndarray) -> np.ndarray:
    """
    Compute Intersection over Union (IoU) of two sets of bounding boxes -
        `boxes_true` and `boxes_detection`. Both sets
        of boxes are expected to be in `(x_min, y_min, x_max, y_max)` format.

    Args:
        boxes_true (np.ndarray): 2D `np.ndarray` representing ground-truth boxes.
            `shape = (N, 4)` where `N` is number of true objects.
        boxes_detection (np.ndarray): 2D `np.ndarray` representing detection boxes.
            `shape = (M, 4)` where `M` is number of detected objects.

    Returns:
        np.ndarray: Pairwise IoU of boxes from `boxes_true` and `boxes_detection`.
            `shape = (N, M)` where `N` is number of true objects and
            `M` is number of detected objects.
    """

    def box_area(box):
        return (box[2] - box[0]) * (box[3] - box[1])

    area_true = box_area(boxes_true.T)
    area_detection = box_area(boxes_detection.T)

    top_left = np.maximum(boxes_true[:, None, :2], boxes_detection[:, :2])
    bottom_right = np.minimum(boxes_true[:, None, 2:], boxes_detection[:, 2:])

    area_inter = np.prod(np.clip(bottom_right - top_left, a_min=0, a_max=None), 2)
    return area_inter / (area_true[:, None] + area_detection - area_inter)

In [74]:
import supervision as sv
import ultralytics
box_annotator = sv.BoxAnnotator()

CONFIDENCE_TRESHOLD = 0.5
predictions = {}

result = list(best_model.predict("/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train/images/100_jpg.rf.08010ef293b05e01c68a3cba517d5260.jpg", conf=CONFIDENCE_TRESHOLD))[0]
detections = sv.Detections(
    xyxy=ultralytics.utils.ops.xyxy2xywh(result.prediction.bboxes_xyxy),
    confidence=result.prediction.confidence,
    class_id=result.prediction.labels.astype(int)
)
#print(detections.xyxy)

#iou(detections_list[0], prediction_list[0])
print(detections_list[49], prediction_list[49])
print(len(detections_list), len(prediction_list))

[2024-01-03 23:55:49] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


[      112.5      149.29      138.57      196.43] Detections(xyxy=array([[     35.758,      64.249,      65.684,      118.41],
       [     112.48,      148.77,      138.77,      196.27]], dtype=float32), mask=None, confidence=array([    0.94629,     0.91455], dtype=float16), class_id=array([0, 0]), tracker_id=None)
315 301
